NAMA : REZA PUTRI ANGGA
NPM : 22083010006
MATA KULIAH : BIG DATA (A)

# RTM-5 AUTOMATED SCORING SYSTEM MENGGUNAKAN PYSPARK

Membuat kode script PySpark untuk melakukan sistem penskoran secara otomatis pada soal essay. Dataset yang digunakan sebagai data latih
diambil dari hasil ujian siswa/mahasiswa dengan sejumlah soal tertentu. Algoritma yang digunakan untuk prediksi menggunakan Alternating Least Square. Output dariprogram berupa akurasi dan pengujian menggunakan data uji soal baru.

In [43]:
!pip install pyspark

# Mengimport Modul Dan Membuat Session

In [1]:
#mengimport modul yang dibutuhkan
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

#membuat session
appName = "Automated Scoring System ALS"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/05/12 05:20:32 WARN Utils: Your hostname, Eza-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.5 instead (on interface en0)
24/05/12 05:20:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/12 05:20:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Membaca Dataset

In [2]:
import pandas as pd

#dikarenakan pyspark tidak mendukung pembacaan langsung dari format file excel
#baca file excel menggunakan pandas
data_pandas = pd.read_excel("training_data_essay.xlsx")

#buat dataframe spark dari dataframe pandas
data = spark.createDataFrame(data_pandas)

In [3]:
#membaca dataset
data.show(10)

+---+------------+--------------------+----+-------------+
|npm|nama_peserta|             jawaban|soal|skor_per_soal|
+---+------------+--------------------+----+-------------+
|  0|       Admin|Tidak, Hanya memb...|   1|        100.0|
|  0|       Admin|Biaya dihitung be...|   2|        100.0|
|  0|       Admin|Hak cipta adalah ...|   3|        100.0|
|  0|       Admin|Dijelaskan kepada...|   4|        100.0|
|  0|       Admin|1. Melindungi dan...|   5|        100.0|
|  0|       Admin|Ruang Komputer, P...|   6|        100.0|
|  0|       Admin|Aturlah posisi pe...|   7|        100.0|
|  0|       Admin|Posisi Kepala dan...|   8|        100.0|
|  0|       Admin|1. Kecocokan soft...|   9|        100.0|
|  0|       Admin|1. Fokus dan expo...|  10|        100.0|
+---+------------+--------------------+----+-------------+
only showing top 10 rows



# Pembersihan Dan Penyiapan Data

Pembersihan Data

In [4]:
from pyspark.sql.functions import count, when, col

#cek missing value
missing_count = data.select([count(when(col(c).isNull(), c)).alias(c) for c in data.columns])
missing_count.show()

+---+------------+-------+----+-------------+
|npm|nama_peserta|jawaban|soal|skor_per_soal|
+---+------------+-------+----+-------------+
|  0|           0|      0|   0|            0|
+---+------------+-------+----+-------------+



In [5]:
from pyspark.sql.functions import count

#cek data duplikat
duplicate_row = data.groupBy(data.columns).count().filter("count > 1")
duplicate_row.show()

+---+------------+-------+----+-------------+-----+
|npm|nama_peserta|jawaban|soal|skor_per_soal|count|
+---+------------+-------+----+-------------+-----+
+---+------------+-------+----+-------------+-----+



Penyiapan Data

In [6]:
#memilih data yang ingin ditampilkan
#kolom "npm", "jawaban", "soal", dan mengubah "skor_per_soal" menjadi "true score"
data = data.select("npm", "jawaban", "soal", col("skor_per_soal").cast("Int").alias("true score"))

data.show(10)

+---+--------------------+----+----------+
|npm|             jawaban|soal|true score|
+---+--------------------+----+----------+
|  0|Tidak, Hanya memb...|   1|       100|
|  0|Biaya dihitung be...|   2|       100|
|  0|Hak cipta adalah ...|   3|       100|
|  0|Dijelaskan kepada...|   4|       100|
|  0|1. Melindungi dan...|   5|       100|
|  0|Ruang Komputer, P...|   6|       100|
|  0|Aturlah posisi pe...|   7|       100|
|  0|Posisi Kepala dan...|   8|       100|
|  0|1. Kecocokan soft...|   9|       100|
|  0|1. Fokus dan expo...|  10|       100|
+---+--------------------+----+----------+
only showing top 10 rows



In [7]:
#pemrosesan data untuk mengubah "jawaban" menjadi vektor fitur
#agar dapat dipahami oleh model ALS
#tokenizing jawaban
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

tokenizer = Tokenizer(inputCol = "jawaban", outputCol = "feature1")
tokenized = tokenizer.transform(data)

tokenized.show(10)

+---+--------------------+----+----------+--------------------+
|npm|             jawaban|soal|true score|            feature1|
+---+--------------------+----+----------+--------------------+
|  0|Tidak, Hanya memb...|   1|       100|[tidak,, hanya, m...|
|  0|Biaya dihitung be...|   2|       100|[biaya, dihitung,...|
|  0|Hak cipta adalah ...|   3|       100|[hak, cipta, adal...|
|  0|Dijelaskan kepada...|   4|       100|[dijelaskan, kepa...|
|  0|1. Melindungi dan...|   5|       100|[1., melindungi, ...|
|  0|Ruang Komputer, P...|   6|       100|[ruang, komputer,...|
|  0|Aturlah posisi pe...|   7|       100|[aturlah, posisi,...|
|  0|Posisi Kepala dan...|   8|       100|[posisi, kepala, ...|
|  0|1. Kecocokan soft...|   9|       100|[1., kecocokan, s...|
|  0|1. Fokus dan expo...|  10|       100|[1., fokus, dan, ...|
+---+--------------------+----+----------+--------------------+
only showing top 10 rows



In [8]:
#menghapus kata2 yang tidak diinginkan dari teks
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

locale = sc._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol = "feature2", locale = "en")
tokenized = swr.transform(tokenized)

tokenized.show(10)

+---+--------------------+----+----------+--------------------+--------------------+
|npm|             jawaban|soal|true score|            feature1|            feature2|
+---+--------------------+----+----------+--------------------+--------------------+
|  0|Tidak, Hanya memb...|   1|       100|[tidak,, hanya, m...|[tidak,, hanya, m...|
|  0|Biaya dihitung be...|   2|       100|[biaya, dihitung,...|[biaya, dihitung,...|
|  0|Hak cipta adalah ...|   3|       100|[hak, cipta, adal...|[hak, cipta, adal...|
|  0|Dijelaskan kepada...|   4|       100|[dijelaskan, kepa...|[dijelaskan, kepa...|
|  0|1. Melindungi dan...|   5|       100|[1., melindungi, ...|[1., melindungi, ...|
|  0|Ruang Komputer, P...|   6|       100|[ruang, komputer,...|[ruang, komputer,...|
|  0|Aturlah posisi pe...|   7|       100|[aturlah, posisi,...|[aturlah, posisi,...|
|  0|Posisi Kepala dan...|   8|       100|[posisi, kepala, ...|[posisi, kepala, ...|
|  0|1. Kecocokan soft...|   9|       100|[1., kecocokan, s...|[1

In [9]:
#mengubah feature jawaban menjadi vektor
from pyspark.ml.feature import VectorAssembler

#mengubah featire 2 menjadi angka 
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol = "feature3")
tokenized = hashTF.transform(tokenized)

tokenized.show(10)

+---+--------------------+----+----------+--------------------+--------------------+--------------------+
|npm|             jawaban|soal|true score|            feature1|            feature2|            feature3|
+---+--------------------+----+----------+--------------------+--------------------+--------------------+
|  0|Tidak, Hanya memb...|   1|       100|[tidak,, hanya, m...|[tidak,, hanya, m...|(262144,[22138,79...|
|  0|Biaya dihitung be...|   2|       100|[biaya, dihitung,...|[biaya, dihitung,...|(262144,[18111,56...|
|  0|Hak cipta adalah ...|   3|       100|[hak, cipta, adal...|[hak, cipta, adal...|(262144,[462,1515...|
|  0|Dijelaskan kepada...|   4|       100|[dijelaskan, kepa...|[dijelaskan, kepa...|(262144,[1532,686...|
|  0|1. Melindungi dan...|   5|       100|[1., melindungi, ...|[1., melindungi, ...|(262144,[10768,10...|
|  0|Ruang Komputer, P...|   6|       100|[ruang, komputer,...|[ruang, komputer,...|(262144,[78139,11...|
|  0|Aturlah posisi pe...|   7|       100|[atu

In [10]:
#mengubah feature jawaban yang telah ditokenized menjadi vektor tunggal dan memilih kolom yang ingin ditampilkan
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors

#fungsi UDF untuk mengekstrak nilai tunggal dari vektor
extract_value = udf(lambda v: float(v[0]), DoubleType())

#membuat kolom baru dengan nilai tunggal dari vektor
tokenized = tokenized.withColumn("features", extract_value("feature3"))

selected_features = ["npm", "soal", "jawaban", "features", "true score"]
data = tokenized.select(selected_features)

data.show(10)

+---+----+--------------------+--------+----------+
|npm|soal|             jawaban|features|true score|
+---+----+--------------------+--------+----------+
|  0|   1|Tidak, Hanya memb...|     0.0|       100|
|  0|   2|Biaya dihitung be...|     0.0|       100|
|  0|   3|Hak cipta adalah ...|     0.0|       100|
|  0|   4|Dijelaskan kepada...|     0.0|       100|
|  0|   5|1. Melindungi dan...|     0.0|       100|
|  0|   6|Ruang Komputer, P...|     0.0|       100|
|  0|   7|Aturlah posisi pe...|     0.0|       100|
|  0|   8|Posisi Kepala dan...|     0.0|       100|
|  0|   9|1. Kecocokan soft...|     0.0|       100|
|  0|  10|1. Fokus dan expo...|     0.0|       100|
+---+----+--------------------+--------+----------+
only showing top 10 rows



# Pembagian Data Training Dan Data Testing

In [25]:
#membagi data dalam dataset menjadi data training 70% dan data testing 30%
split_data = data.randomSplit([0.7, 0.3])
training_data = split_data[0]
testing_data = split_data[1]

#jumlah data training dan data testing
training_row = training_data.count()
testing_row = testing_data.count()

print("Jumlah Data Training : ", training_row, " Dan Jumlah Data Testing : ", testing_row)

Jumlah Data Training :  84  Dan Jumlah Data Testing :  36


In [26]:
#menampilkan data testing
training_data.show(84)

+----------+----+--------------------+--------+----------+
|       npm|soal|             jawaban|features|true score|
+----------+----+--------------------+--------+----------+
|         0|   1|Tidak, Hanya memb...|     0.0|       100|
|         0|   2|Biaya dihitung be...|     0.0|       100|
|         0|   4|Dijelaskan kepada...|     0.0|       100|
|         0|   5|1. Melindungi dan...|     0.0|       100|
|         0|   8|Posisi Kepala dan...|     0.0|       100|
|         0|   9|1. Kecocokan soft...|     0.0|       100|
|         0|  10|1. Fokus dan expo...|     0.0|       100|
|         0|  11|1. Peralatan yang...|     0.0|       100|
|1121020033|   1|tidak, cuma mengi...|     0.0|        52|
|1121020033|   2|biaya dihitung be...|     0.0|        42|
|1121020033|   3|hak membuat merup...|     0.0|        42|
|1121020033|   4|dipaparkan pada k...|     0.0|        27|
|1121020033|   5|1. mencegah serta...|     0.0|        44|
|1121020033|   6|ruang komputer, p...|     0.0|       10

In [27]:
#menampilkan data training
testing_data.show(36)

+----------+----+--------------------+--------+----------+
|       npm|soal|             jawaban|features|true score|
+----------+----+--------------------+--------+----------+
|         0|   3|Hak cipta adalah ...|     0.0|       100|
|         0|   6|Ruang Komputer, P...|     0.0|       100|
|         0|   7|Aturlah posisi pe...|     0.0|       100|
|         0|  12|1. Dibuat grafik ...|     0.0|       100|
|1121020033|   7|aturlah posisi fi...|     0.0|        57|
|1121020033|   9|1.kesesuaian apli...|     0.0|        54|
|1121020033|  12|metode digital ar...|     0.0|        26|
|1220020029|   5|melindungi dan me...|     0.0|        74|
|1220020018|   6|ruang komputer, p...|     0.0|       100|
|1220020029|   9|kecocokan softwar...|     0.0|        84|
|1220020029|  12|dibuat grafik yan...|     0.0|        81|
|1220020018|  12|dibuat grafik yan...|     0.0|        86|
|1220020023|   1|tidak, hanya memb...|     0.0|       100|
|1220020023|   3|hak cipta adalah ...|     0.0|        9

# Pembangunan Model ALS (Alternating Least Square)

In [28]:
#membangun model ALS
from pyspark.ml.feature import StringIndexer
from pyspark.ml.recommendation import ALS

#karena semua kolom sudah bertipe numerik
#dan memenuhi syarat pembangunan model ALS, maka dibangun model ALS
als = ALS(maxIter = 10, regParam = 0.01, userCol = "soal", itemCol = "features", 
          ratingCol = "true score")

#melatih model als
model = als.fit(training_data)

24/05/12 05:21:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/05/12 05:21:16 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


# Pengujian Model Berdasarkan Model ALS (Alternating Least Square) Menggunakan Data Testing

In [29]:
#lakukan prediksi dan penampilan pada data pengujian
from pyspark.sql.functions import round

predictions_testing_data = model.transform(testing_data)
predictions_testing_data = predictions_testing_data.withColumn("prediction", 
                                                               round(predictions_testing_data["prediction"], 2))

predictions_testing_data.show(36)

+----------+----+--------------------+--------+----------+----------+
|       npm|soal|             jawaban|features|true score|prediction|
+----------+----+--------------------+--------+----------+----------+
|         0|  12|1. Dibuat grafik ...|     0.0|       100|     71.33|
|         0|   6|Ruang Komputer, P...|     0.0|       100|     100.0|
|         0|   3|Hak cipta adalah ...|     0.0|       100|      78.2|
|         0|   7|Aturlah posisi pe...|     0.0|       100|      88.0|
|1121020033|  12|metode digital ar...|     0.0|        26|     71.33|
|1220020029|   5|melindungi dan me...|     0.0|        74|     76.62|
|1121020033|   9|1.kesesuaian apli...|     0.0|        54|     80.29|
|1121020033|   7|aturlah posisi fi...|     0.0|        57|      88.0|
|1220020029|  12|dibuat grafik yan...|     0.0|        81|     71.33|
|1220020018|   6|ruang komputer, p...|     0.0|       100|     100.0|
|1220020029|   9|kecocokan softwar...|     0.0|        84|     80.29|
|1220020018|  12|dib

# Akurasi Model

In [30]:
from pyspark.ml.evaluation import RegressionEvaluator

#perhitungan RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "true score", predictionCol = "prediction")
rmse = evaluator.evaluate(predictions_testing_data)

#menampilkan hasil
print("Root Mean Square Error (RMSE) : ", rmse)

Root Mean Square Error (RMSE) :  18.896240372702476


In [31]:
correct_predictions = predictions_testing_data.filter(predictions_testing_data["prediction"] == predictions_testing_data["true score"])

total_data = predictions_testing_data.count()
correct_count = correct_predictions.count()  #menghitung jumlah prediksi yang benar

accuracy = correct_count / total_data  #menghitung akurasi

print("Prediksi Benar : ", correct_count, 
      "Total Data : ", total_data,
      "Akurasi : ", accuracy)

Prediksi Benar :  4 Total Data :  36 Akurasi :  0.1111111111111111


# Data Uji Soal Baru

In [35]:
import pandas as pd
import random

# Data uji baru yang ingin dibuat
data_uji = {
    "npm": ["1121020006"] * 12,  #semua npm adalah '06'
    "soal": list(range(1, 13)),  #kode soal dari 1 hingga 12
    "jawaban": [
        "Satu software sudah cukup karena sesuai dengan keahlian yang dibutuhkan.",
        "Perhitungan biaya didasarkan pada waktu pengerjaan dan tingkat kesulitan proyek.",
        "Hak cipta merupakan hak eksklusif bagi pencipta atau penerima hak untuk mengumumkan atau memperbanyak karyanya, dengan memperhatikan peraturan perundang-undangan yang berlaku.",
        "Penting untuk menjelaskan kepada klien bahwa ukuran huruf yang terlalu besar atau terlalu kecil dapat memengaruhi kenyamanan pembaca.",
        "Langkah-langkah untuk melindungi dan menjaga keamanan di tempat kerja mencakup berbagai hal, seperti hak keselamatan pekerja dan pemeliharaan sumber daya produksi.",
        "Ruang komputer harus diatur sedemikian rupa agar nyaman digunakan, termasuk posisi tubuh dan posisi komputer yang tepat.",
        "Untuk mengurangi ketegangan otot dan pikiran, penting untuk melakukan istirahat yang cukup dan gerakan tubuh secara teratur saat bekerja di depan komputer.",
        "Untuk memastikan kenyamanan saat bekerja, penting untuk menjaga posisi kepala, leher, punggung, pundak, lengan, dan siku dalam posisi yang tepat.",
        "Saat memilih perangkat lunak kamera, perlu dipertimbangkan kesesuaian dengan perangkat keras, resolusi kamera, dan kapasitas RAM untuk memastikan kualitas gambar yang dihasilkan.",
        "Fokus dan eksposur kamera digital harus diperhatikan agar gambar yang dihasilkan sesuai dengan kebutuhan, termasuk penggunaan perangkat lunak gambar yang tepat dan pengaturan file gambar yang disimpan.",
        "Saat mentransfer gambar, perlu diidentifikasi peralatan dan materi yang diperlukan, serta memastikan peralatan teknis dioperasikan sesuai standar untuk transfer yang benar.",
        "Desain grafis harus memperhatikan prinsip desain visual, teknik digital artwork, dan penggunaan software yang tepat untuk menghasilkan grafik yang sesuai dengan kebutuhan."
    ],
    "true score": [random.uniform(60, 100) for _ in range(12)]  # Skor acak antara 60 dan 100
}

#membuat DataFrame dari data uji
data_uji_baru = pd.DataFrame(data_uji)

# Menampilkan DataFrame
data_uji_baru

,npm,soal,jawaban,true score
0,1121020006,1,Satu software sudah cukup karena sesuai dengan...,70.752525
1,1121020006,2,Perhitungan biaya didasarkan pada waktu penger...,60.919572
2,1121020006,3,Hak cipta merupakan hak eksklusif bagi pencipt...,71.858525
3,1121020006,4,Penting untuk menjelaskan kepada klien bahwa u...,73.820298
4,1121020006,5,Langkah-langkah untuk melindungi dan menjaga k...,92.810806
5,1121020006,6,Ruang komputer harus diatur sedemikian rupa ag...,80.650687
6,1121020006,7,"Untuk mengurangi ketegangan otot dan pikiran, ...",80.319357
7,1121020006,8,"Untuk memastikan kenyamanan saat bekerja, pent...",86.776008
8,1121020006,9,"Saat memilih perangkat lunak kamera, perlu dip...",97.714766
9,1121020006,10,Fokus dan eksposur kamera digital harus diperh...,69.376626


In [37]:
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

#membuat sesi spark untuk mengubah data uji baru menjadi vektor
spark = SparkSession.builder \
    .appName("Data Uji Baru") \
    .getOrCreate()

#membuat DataFrame PySpark dari data uji
data_uji_spark = spark.createDataFrame(data_uji_baru)

#tokenisasi jawaban
tokenizer = Tokenizer(inputCol="jawaban", outputCol="words")
wordsData = tokenizer.transform(data_uji_spark)

#menggunakan HashingTF untuk mengubah kata-kata menjadi fitur vektor
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

#menghitung IDF untuk mendapatkan vektor akhir
idf = IDF(inputCol="rawFeatures", outputCol="jawaban_vektor")
idfModel = idf.fit(featurizedData)
data_vectorized = idfModel.transform(featurizedData)

#pemilihan kolom yang diperlukan untuk model ALS
data_selected = data_vectorized.select("npm", "soal", "jawaban", "true score", "jawaban_vektor")

#fungsi UDF untuk mengekstrak nilai tunggal dari vektor
extract_value = udf(lambda v: float(v[0]), DoubleType())

#membuat kolom baru dengan nilai tunggal dari vektor
data_selected = data_selected.withColumn("features", extract_value("jawaban_vektor"))

#menampilkan DataFrame yang sudah diubah ke dalam vektor
data_final = data_selected.select("npm", "soal", "jawaban", "features", "true score")
data_final.show()

+----------+----+--------------------+--------+-----------------+
|       npm|soal|             jawaban|features|       true score|
+----------+----+--------------------+--------+-----------------+
|1121020006|   1|Satu software sud...|     0.0| 70.7525245899488|
|1121020006|   2|Perhitungan biaya...|     0.0|60.91957185793961|
|1121020006|   3|Hak cipta merupak...|     0.0|71.85852543131415|
|1121020006|   4|Penting untuk men...|     0.0|73.82029818135986|
|1121020006|   5|Langkah-langkah u...|     0.0| 92.8108063809619|
|1121020006|   6|Ruang komputer ha...|     0.0|80.65068661981215|
|1121020006|   7|Untuk mengurangi ...|     0.0|80.31935657019821|
|1121020006|   8|Untuk memastikan ...|     0.0|86.77600833946627|
|1121020006|   9|Saat memilih pera...|     0.0|97.71476565028138|
|1121020006|  10|Fokus dan eksposu...|     0.0|69.37662586918414|
|1121020006|  11|Saat mentransfer ...|     0.0|84.79248130763396|
|1121020006|  12|Desain grafis har...|     0.0|71.39293374117395|
+---------

In [39]:
#lakukan prediksi dan penampilan pada data pengujian
from pyspark.sql.functions import round

predictions_testing_data_new = model.transform(data_final)
predictions_testing_data_new = predictions_testing_data_new.withColumn("prediction", round(predictions_testing_data_new
                                                                                           ["prediction"], 2))

predictions_testing_data_new.show()

+----------+----+--------------------+--------+-----------------+----------+
|       npm|soal|             jawaban|features|       true score|prediction|
+----------+----+--------------------+--------+-----------------+----------+
|1121020006|   1|Satu software sud...|     0.0| 70.7525245899488|      94.0|
|1121020006|   3|Hak cipta merupak...|     0.0|71.85852543131415|      78.2|
|1121020006|   2|Perhitungan biaya...|     0.0|60.91957185793961|     83.56|
|1121020006|   4|Penting untuk men...|     0.0|73.82029818135986|     68.12|
|1121020006|   6|Ruang komputer ha...|     0.0|80.65068661981215|     100.0|
|1121020006|   5|Langkah-langkah u...|     0.0| 92.8108063809619|     76.62|
|1121020006|   7|Untuk mengurangi ...|     0.0|80.31935657019821|      88.0|
|1121020006|   9|Saat memilih pera...|     0.0|97.71476565028138|     80.29|
|1121020006|   8|Untuk memastikan ...|     0.0|86.77600833946627|      82.5|
|1121020006|  10|Fokus dan eksposu...|     0.0|69.37662586918414|     85.14|

# Akurasi Model Data Uji Soal Baru

In [40]:
from pyspark.ml.evaluation import RegressionEvaluator

#perhitungan RMSE
evaluator = RegressionEvaluator(metricName = "rmse", labelCol = "true score", predictionCol = "prediction")
rmse = evaluator.evaluate(predictions_testing_data_new)

#menampilkan hasil
print("Root Mean Square Error (RMSE) : ", rmse)

Root Mean Square Error (RMSE) :  14.128128002543683


In [42]:
correct_predictions = predictions_testing_data_new.filter(predictions_testing_data_new["prediction"] == predictions_testing_data_new["true score"])

total_data = predictions_testing_data_new.count()
correct_count = correct_predictions.count()  #menghitung jumlah prediksi yang benar

accuracy = correct_count / total_data  #menghitung akurasi

print("Prediksi Benar : ", correct_count, 
      "Total Data : ", total_data,
      "Akurasi : ", accuracy)

Prediksi Benar :  0 Total Data :  12 Akurasi :  0.0


# Automated Scoring System Menggunakan PySpark Dengan Hash

In [44]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import hash
import pandas as pd

#inisialisasi SparkSession
spark = SparkSession.builder \
    .appName("Automated Scoring System ALS (Hash)") \
    .getOrCreate()

#baca file excel menggunakan pandas dan buat dataframe Spark
data = spark.createDataFrame(pd.read_excel("training_data_essay.xlsx"))

#ubah jawaban menjadi vektor hash
hashed_data = data.withColumn("vektor hash", hash("jawaban")).select("npm", "soal", "vektor hash", 
                                                                     col("skor_per_soal").alias("true score"))

#membagi data menjadi data latih dan data uji
(training_data, testing_data) = hashed_data.randomSplit([0.7, 0.3])
print("Jumlah Data Training:", training_data.count())
print("Jumlah Data Training:", testing_data.count())

#membuat dan membangun model ALS
model = ALS(maxIter=10, regParam=0.1, userCol="soal", itemCol="vektor hash", ratingCol="true score").fit(
    training_data)

#memprediksi skor pada data uji
predictions = model.transform(testing_data)

#evaluasi kinerja model dengan RMSE
rmse = RegressionEvaluator(metricName="rmse", labelCol="true score", predictionCol="prediction").evaluate(
    predictions)

#menampilkan hasil hash
hashed_data.select("npm", "soal", "vektor hash", "true score").show(truncate=False)

24/05/12 09:38:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Jumlah Data Training: 84
Jumlah Data Training: 36
+----------+----+-----------+----------+
|npm       |soal|vektor hash|true score|
+----------+----+-----------+----------+
|0         |1   |-2059296905|100.0     |
|0         |2   |1183180174 |100.0     |
|0         |3   |1232762403 |100.0     |
|0         |4   |-2035408785|100.0     |
|0         |5   |1588395990 |100.0     |
|0         |6   |339970513  |100.0     |
|0         |7   |50850002   |100.0     |
|0         |8   |-945877996 |100.0     |
|0         |9   |1576366224 |100.0     |
|0         |10  |-1905649442|100.0     |
|0         |11  |550139146  |100.0     |
|0         |12  |1727767227 |100.0     |
|1121020033|1   |1947733435 |52.7      |
|1121020033|2   |-1139863335|42.86     |
|1121020033|3   |122676417  |42.16     |
|1121020033|4   |-1054163002|27.19     |
|1121020033|5   |1990940339 |44.14     |
|1121020033|6   |1770907636 |100.0     |
|1121020033|7   |-463479969 |57.68     |
|1121020033|8   |-412537011 |45.71     |
+------

In [52]:
#menampilkan hasil prediksi pada data testing
from pyspark.sql.functions import hash, col, format_number  

predictions.select("npm", "soal", "vektor hash", "true score", 
                   format_number("prediction", 2).alias("prediction")).show(36)

+----------+----+-----------+----------+----------+
|       npm|soal|vektor hash|true score|prediction|
+----------+----+-----------+----------+----------+
|         0|   1|-2059296905|     100.0|       NaN|
|         0|   5| 1588395990|     100.0|       NaN|
|         0|   9| 1576366224|     100.0|       NaN|
|         0|   8| -945877996|     100.0|       NaN|
|         0|   7|   50850002|     100.0|       NaN|
|         0|  11|  550139146|     100.0|       NaN|
|         0|   2| 1183180174|     100.0|       NaN|
|1121020033|   2|-1139863335|     42.86|       NaN|
|1121020033|   6| 1770907636|     100.0|    100.00|
|1220020029|   6| 1770907636|     100.0|    100.00|
|1220020029|   3|  770340049|     91.71|     91.71|
|1121020033|   5| 1990940339|     44.14|       NaN|
|1220020018|   5|-1932865057|     74.73|     74.73|
|1220020018|   9|-1019902387|     99.31|       NaN|
|1220020029|   9|-1092404005|     84.88|     84.87|
|1220020029|  10|-1672280317|     100.0|       NaN|
|1220020029|

In [53]:
#akurasi data
#rmse
print("Root Mean Squared Error (RMSE) pada data uji:", rmse)

Root Mean Squared Error (RMSE) pada data uji: nan


In [55]:
#menghapus baris yang memiliki nilai prediksi nan agar bisa dihitung rmse
predictions_without_nan = predictions.na.drop(subset=["prediction"])
predictions_without_nan.select("npm", "soal", "vektor hash", "true score", 
                               format_number("prediction", 2).alias("prediction")).show()

+----------+----+-----------+----------+----------+
|       npm|soal|vektor hash|true score|prediction|
+----------+----+-----------+----------+----------+
|1121020033|   6| 1770907636|     100.0|    100.00|
|1220020029|   6| 1770907636|     100.0|    100.00|
|1220020029|   3|  770340049|     91.71|     91.71|
|1220020018|   5|-1932865057|     74.73|     74.73|
|1220020029|   9|-1092404005|     84.88|     84.87|
|1220020018|   2| 1176853507|     100.0|    100.00|
|1220020023|   3|  770340049|     91.71|     91.71|
|1220020023|   9|  447829639|     65.89|     65.89|
|1220020023|   7|-1392782412|     86.22|     86.21|
|1220020023|   2| 1176853507|     100.0|    100.00|
|1120020017|   8| 1044171719|     89.17|     89.17|
|1120020017|  11|-1838887780|     80.09|     80.09|
|1121020036|   1| -256638840|     100.0|    100.00|
|1121020036|  12| -902409772|     86.53|     86.53|
|1121020035|   1| -256638840|     100.0|    100.00|
+----------+----+-----------+----------+----------+



In [58]:
#rmse
evaluator = RegressionEvaluator(metricName="rmse", labelCol="true score", predictionCol="prediction")
rmse = evaluator.evaluate(predictions_without_nan)

print("Root Mean Squared Error (RMSE) Setelah Menghapus NaN : ", rmse)

Root Mean Squared Error (RMSE) Setelah Menghapus NaN :  0.0039636533191873425


In [ ]:
spark.stop()